# Model Quantization
## Quantization Aware Training using Intel Neural Compressor

In [1]:
import os

os.environ["LOGLEVEL"] = "CRITICAL"
import lightning as L
import torch
from lightning.pytorch import seed_everything
from lightning.pytorch.callbacks import EarlyStopping, RichModelSummary, RichProgressBar
from neural_compressor.config import QuantizationAwareTrainingConfig
from neural_compressor.training import fit, prepare_compression

from dataset import PetDataModule
from model import PetClassifier
from utils import print_size_of_model

In [2]:
%reload_ext watermark
%watermark --iversions

torch    : 2.1.0
lightning: 2.1.1



In [3]:
CHECKPOINT = "/home/ababu/mlw_2023/checkpoints/epoch=8-step=720-v1.ckpt"
BATCH_SIZE = 32
NUM_WORKERS = 8
DEVICE = "cpu"
SEED = 42
seed_everything(SEED, workers=True)

Seed set to 42


42

In [4]:
dm = PetDataModule(BATCH_SIZE, NUM_WORKERS)

In [5]:
torch.set_flush_denormal(True)

True

In [6]:
def train_func(model_n):
    setattr(base_model, "model", model_n)
    bar = RichProgressBar()
    early_stopping = EarlyStopping("val_loss", patience=5)
    trainer = L.Trainer(accelerator="gpu", callbacks=[bar, early_stopping])
    trainer.fit(base_model, datamodule=dm)
    return base_model

In [7]:
conf = QuantizationAwareTrainingConfig()

lr = 1e-4
weight_decay = 1e-6
num_classes = 37
base_model = PetClassifier(num_classes, lr, weight_decay)

compression_manager = prepare_compression(base_model, conf)
compression_manager.callbacks.on_train_begin()
qmodel = compression_manager.model
train_func(qmodel)
compression_manager.callbacks.on_train_end()
compression_manager.save("./qat_output")

/home/ababu/mlw_2023/.venv1/lib/python3.10/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/home/ababu/mlw_2023/.venv1/lib/python3.10/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/home/ababu/mlw_2023/.venv1/lib/python3.10/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/home/ababu/mlw_2023/.venv1/lib/python3.10/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,
/home/ababu/mlw_2023/.venv1/lib/python3.10/site-packages/torch/ao/quantization/fx/fuse.py:56: UserWarning: Passing a fuse_custom_config_dict to fuse is deprecated and will not be supported in a future version. Please pass in a FuseCustomConfig instead.
  war

┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name           ┃ Type               ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ loss_module    │ CrossEntropyLoss   │      0 │
│ 1 │ model          │ PyTorchFXModel     │  4.5 M │
│ 2 │ train_accuracy │ MulticlassAccuracy │      0 │
│ 3 │ val_accuracy   │ MulticlassAccuracy │      0 │
│ 4 │ test_accuracy  │ MulticlassAccuracy │      0 │
└───┴────────────────┴────────────────────┴────────┘

Trainable params: 4.5 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.5 M                                                                                                
Total estimated model params size (MB): 18

Output()

/home/ababu/mlw_2023/.venv1/lib/python3.10/site-packages/torch/ao/quantization/fake_quantize.py:343: UserWarning: 
_aminmax is deprecated as of PyTorch 1.11 and will be removed in a future release. Use aminmax instead. This 
warning will only appear once per process. (Triggered internally at ../aten/src/ATen/native/ReduceAllOps.cpp:72.)
  return torch.fused_moving_avg_obs_fake_quant(

/home/ababu/mlw_2023/.venv1/lib/python3.10/site-packages/torch/ao/quantization/fake_quantize.py:343: UserWarning: 
_aminmax is deprecated as of PyTorch 1.11 and will be removed in a future release. Use aminmax instead. This 
warning will only appear once per process. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:677.)
  return torch.fused_moving_avg_obs_fake_quant(

In [8]:
bar = RichProgressBar()
summary = RichModelSummary()
trainer = L.Trainer(accelerator=DEVICE, callbacks=[bar, summary])

Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/ababu/mlw_2023/.venv1/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


In [9]:
setattr(base_model, "model", qmodel.model)
trainer.test(model=base_model, datamodule=dm)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │     0.492324560880661     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc_epoch': 0.492324560880661}]

In [10]:
%%timeit
base_model.eval()
with torch.no_grad():
    base_model(torch.rand((1, 3, 224, 224), dtype=torch.float32))

20.2 ms ± 1.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
print_size_of_model(base_model.model)

Size (MB): 2.672386

In [13]:
model = PetClassifier.load_from_checkpoint(CHECKPOINT)
device = torch.device("cpu")
model.to(device)
print_size_of_model(model)

Size (MB): 9.311108

In [15]:
%%timeit
model.eval()
with torch.no_grad():
    model(torch.rand((1, 3, 224, 224), dtype=torch.float32))

25.6 ms ± 471 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
